In [2]:
import numpy as np
from sight_line_constructor import Sightline
import read_cubes
from scipy import ndimage
import matplotlib.pyplot as plt
from astropy.io import ascii



In [60]:
path = "/var/lib/libvirt/images/nbody/baorsd/run101/density_field/"

cube = read_cubes.read(path+"S020_CIC1024_conf.bin")

In [4]:
res = 2
start_point = np.array([0,0,0]) #Mpc/h
end_point = np.array([300,300,500]) #Mpc/h


In [5]:
s1 = Sightline(start_point,end_point,num=3000)

In [6]:
los_dens = s1.calc_los(res,cube)

In [7]:
end_points = np.random.uniform(-2,2,(5,3))

In [10]:
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u

In [93]:
from scipy import interpolate

cosmo = FlatLambdaCDM(H0=100, Om0=0.315)

reds = np.linspace(0.,10.,10000)
dists = cosmo.comoving_distance(reds)

z_dist = interpolate.interp1d(reds, dists)



In [31]:
files = ascii.read('/home/mata/cosmic-sightlines/simlist.txt',names=["filename", 'redshift'])

In [100]:
z_dist(files['redshift'].data)

array([2990.59785349, 2821.67991357, 2656.19952512, 2493.88159244,
       2334.43809439, 2177.61231632, 2023.18108248, 1870.91593272,
       1720.60666364, 1572.0561105 , 1425.06900518, 1279.45875188,
       1135.04359213,  991.63674372,  849.05714337,  707.12042403,
        565.62997617,  424.39312659,  283.19353311,  141.80987524,
          0.        ])

In [77]:
#Function to read in particular file for distance

#first read all files:
n_files = len(files)
nc = 1024
#make an array of all files with corresponding redshift

l_arr = []
for i in range(len(files)):
    tmp = read_cubes.read(path+str(files[i]['filename']))
    l_arr.append(tmp)
l_arr = np.asarray(l_arr)


